# 第二章：明确和直接

## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。

In [1]:
# 安装OpenAI库
%pip install openai==1.61.0

# 🔧 OpenAI环境自动配置
# 此设置会自动从环境变量或IPython存储中加载配置

# 安装OpenAI库
%pip install openai==1.61.0

# 导入Python内置的正则表达式库
import re

# 🚀 使用统一配置管理系统
from config import setup_notebook_environment, print_config_info

# 自动设置OpenAI客户端和get_completion函数
# 优先级：环境变量 > IPython存储 > 默认值
try:
    client, get_completion = setup_notebook_environment()
    print("✅ 使用统一配置管理成功！")
except Exception as e:
    print(f"❌ 统一配置失败，回退到传统方式: {e}")
    
    # 回退到传统的配置方式
    import openai
    
    # 从IPython存储中检索API_KEY和MODEL_NAME变量
    %store -r API_KEY
    %store -r MODEL_NAME

    # 如果没有设置MODEL_NAME，使用默认值
    try:
        MODEL_NAME
    except NameError:
        MODEL_NAME = "gpt-4o"  # 默认使用gpt-4o模型

    # 创建OpenAI客户端
    client = openai.OpenAI(api_key=API_KEY)

    def get_completion(prompt: str, system_prompt=""):
        """
        获取GPT的完成响应
        
        参数:
            prompt (str): 用户提示
            system_prompt (str): 系统提示（可选）
        
        返回:
            str: GPT的响应文本
        """
        # 构建消息列表
        messages = []
        
        # 如果有系统提示，添加系统消息
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        
        # 添加用户消息
        messages.append({"role": "user", "content": prompt})
        
        # 调用OpenAI API
        response = client.chat.completions.create(
            model=MODEL_NAME,              # 模型名称 (gpt-4o 或 deepseek-r1)
            messages=messages,             # 消息列表
            max_completion_tokens=2000,    # 最大token数
            temperature=0.0               # 温度参数，0表示更确定性
        )
        return response.choices[0].message.content
    
    print("⚠️  使用传统配置方式，建议配置环境变量以获得更好体验")

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
✅ OpenAI环境设置完成!
🔧 OpenAI API 配置信息:
  📡 配置来源: 环境变量 + 自定义API地址: https://vip.apiyi.com/v1
  🤖 模型: gpt-4o
  🌐 API地址: https://vip.apiyi.com/v1
  🔑 API密钥: sk-R2utG...B944

✅ 使用统一配置管理成功！


---

## 课程 {#lesson}

**GPT对清晰直接的指令响应最佳。**

把GPT想象成任何一个刚入职的人。除了您直接告诉它的内容外，**GPT对该做什么没有任何上下文**。就像您第一次指导人类完成任务一样，您越是以直接的方式向GPT准确解释您想要什么，GPT的响应就会越好、越准确。"				
				
当有疑问时，遵循**清晰提示的黄金法则**：
- 将您的提示展示给同事或朋友，让他们自己按照指令操作，看看他们能否产生您想要的结果。如果他们感到困惑，GPT也会困惑。				

### 示例

让我们通过几个对比鲜明的例子来演示明确和直接的重要性。

#### 示例1：模糊 vs 明确的任务指令

In [2]:
# 模糊的提示
PROMPT = "帮我处理一下这个数据"

# 显示GPT回答
print("模糊提示的回答:")
print(get_completion(PROMPT))
print()

模糊提示的回答:
当然可以！请提供您需要处理的数据，并告诉我您希望如何处理它。无论是数据清理、分析还是可视化，我都会尽力帮助您。



可以看到，GPT无法理解"处理数据"具体指什么，只能给出通用的回答。

现在让我们提供一个**明确和直接**的指令：

In [3]:
# 明确和直接的提示
PROMPT = """分析以下销售数据，计算每个月的同比增长率：
1月：100万
2月：120万
3月：110万
4月：130万

要求：
- 以表格形式输出结果
- 包含月份、销售额、同比增长率三列
- 同比增长率保留2位小数，用百分比表示"""

# 显示GPT回答
print("明确提示的回答:")
print(get_completion(PROMPT))

明确提示的回答:
为了计算每个月的同比增长率，我们需要假设去年每个月的销售额。由于没有提供去年的数据，我将假设去年每个月的销售额与今年相同，以便展示表格格式和计算方法。

假设去年每个月的销售额如下：
- 去年1月：100万
- 去年2月：120万
- 去年3月：110万
- 去年4月：130万

根据这个假设，计算每个月的同比增长率：

| 月份 | 销售额 (万元) | 同比增长率 |
|------|--------------|------------|
| 1月  | 100          | 0.00%      |
| 2月  | 120          | 0.00%      |
| 3月  | 110          | 0.00%      |
| 4月  | 130          | 0.00%      |

如果有去年的实际数据，请提供，以便进行准确的同比增长率计算。否则，以上表格仅用于展示格式。


看到差别了吗？明确的指令让GPT能够准确理解任务，并按照要求的格式输出结果。

#### 示例2：输出格式的明确指定

让我们看另一个例子，关于如何明确指定输出格式。首先是一个**模糊的请求**：

In [4]:
# 模糊的提示 - 没有指定输出格式
PROMPT = "给我一些Python学习建议"

# 显示GPT回答
print("模糊提示的回答:")
print(get_completion(PROMPT))
print("\n" + "="*50 + "\n")

模糊提示的回答:
学习Python是一项非常有益的技能，无论你是初学者还是有经验的程序员。以下是一些建议，可以帮助你有效地学习Python：

1. **设定明确的目标**：首先，明确你学习Python的目的。是为了数据分析、机器学习、Web开发还是自动化任务？根据你的目标选择合适的学习路径。

2. **掌握基础知识**：确保你理解Python的基本语法和概念，包括变量、数据类型、操作符、条件语句、循环、函数和类等。

3. **使用交互式学习平台**：像Codecademy、LeetCode、HackerRank和Kaggle等平台提供了交互式的Python练习，可以帮助你巩固基础知识。

4. **阅读官方文档**：Python的官方文档是一个非常全面的资源，帮助你深入理解Python的功能和库。

5. **动手实践**：理论学习固然重要，但实践才能让你真正掌握Python。尝试编写小项目，比如简单的计算器、数据分析脚本或网页爬虫。

6. **学习Python库**：根据你的目标，学习相关的Python库。例如，数据分析可以学习Pandas和NumPy，机器学习可以学习Scikit-learn和TensorFlow，Web开发可以学习Flask和Django。

7. **参与社区**：加入Python社区，比如Stack Overflow、Reddit的r/learnpython或GitHub，参与讨论，寻求帮助，分享你的项目。

8. **定期练习**：编程技能需要不断练习。设定一个学习计划，每天或每周定期编写代码，解决问题。

9. **解决实际问题**：尝试解决实际问题或挑战，比如自动化日常任务、分析数据集或开发一个简单的应用程序。

10. **持续学习**：Python生态系统不断发展，保持学习新技术、新库和新工具的习惯。

通过这些建议，你可以逐步提高你的Python技能，并应用到实际项目中。祝你学习愉快！




GPT给出了一般性的建议，但格式不够结构化。现在让我们用**明确的格式要求**：

In [5]:
# 明确和直接的提示 - 指定了具体的输出格式
PROMPT = """为初学者制定一个Python学习计划。

要求：
1. 分为3个阶段：初级、中级、高级
2. 每个阶段包含：学习时长、核心概念、推荐资源、实战项目
3. 用表格形式呈现
4. 不要添加额外解释，直接给出表格

输出格式示例：
| 阶段 | 学习时长 | 核心概念 | 推荐资源 | 实战项目 |
|------|----------|----------|----------|----------|
| 初级 | ... | ... | ... | ... |"""

# 显示GPT回答
print("明确提示的回答:")
print(get_completion(PROMPT))

明确提示的回答:
| 阶段 | 学习时长 | 核心概念 | 推荐资源 | 实战项目 |
|------|----------|----------|----------|----------|
| 初级 | 4周 | 变量与数据类型、基本运算符、条件语句、循环 | 《Python编程：从入门到实践》、Codecademy Python课程 | 计算器程序、猜数字游戏 |
| 中级 | 6周 | 函数、列表与字典、文件操作、错误和异常处理 | 《流畅的Python》、LeetCode编程题 | 图书管理系统、数据分析脚本 |
| 高级 | 8周 | 面向对象编程、模块与包、网络编程、数据库操作 | 《Python高级编程》、Coursera Python课程 | Web应用开发、自动化脚本工具 |


#### 示例3：避免歧义的明确指令

最后一个例子展示如何通过明确指令避免歧义。先看一个**容易产生歧义**的提示：


In [6]:
# 容易产生歧义的提示
PROMPT = "解释一下机器学习"

# 显示GPT回答
print("歧义提示的回答:")
print(get_completion(PROMPT))
print("\n" + "="*50 + "\n")


歧义提示的回答:
机器学习是一种通过算法和统计模型使计算机系统能够从数据中自动学习和改进的技术。它是人工智能的一个分支，旨在开发能够从经验中学习的系统，而无需明确编程。机器学习的核心思想是利用数据来训练模型，使其能够识别模式、做出预测或决策。

机器学习可以分为几种主要类型：

1. **监督学习**：在这种类型中，模型从标记的数据中学习。数据集包含输入和对应的正确输出，模型通过学习输入与输出之间的关系来进行预测。常见的监督学习任务包括分类（如垃圾邮件检测）和回归（如房价预测）。

2. **无监督学习**：模型从未标记的数据中寻找模式或结构。常见的无监督学习任务包括聚类（如客户分群）和降维（如主成分分析）。

3. **半监督学习**：结合了监督学习和无监督学习，使用少量标记数据和大量未标记数据进行训练。这种方法在标记数据难以获取时特别有用。

4. **强化学习**：模型通过与环境交互来学习策略，以最大化某种奖励。常用于机器人控制、游戏和自动驾驶等领域。

机器学习的应用非常广泛，包括图像识别、自然语言处理、推荐系统、金融预测、医疗诊断等。随着数据量的增加和计算能力的提升，机器学习在各个领域的影响力和应用范围不断扩大。




In [7]:
# 明确的上下文和要求
PROMPT = """你是一名面向大学一年级计算机专业学生的讲师。用简单易懂的语言解释机器学习。

要求：
1. 用日常生活中的例子来说明概念
2. 避免复杂的数学公式
3. 重点说明机器学习的3个主要类型及其应用场景
4. 控制在200字以内
5. 用1-2-3的序号格式组织内容"""

# 显示GPT回答
print("明确提示的回答:")
print(get_completion(PROMPT))


明确提示的回答:
1. **监督学习**：想象你在学习如何识别水果。你的老师给你一篮子标有名称的水果（苹果、香蕉等），你通过观察这些标记来学习识别不同的水果。这就是监督学习，机器通过已知的输入和输出对来学习模式。应用场景包括垃圾邮件过滤和图像识别。

2. **无监督学习**：假设你有一篮子没有标签的水果，你需要自己找出哪些水果相似。无监督学习就是这样，机器在没有明确答案的情况下寻找数据中的模式。常用于客户分群和推荐系统。

3. **强化学习**：想象你在玩电子游戏，通过不断尝试和失败来学习如何过关。强化学习就是这样，机器通过试错和从环境中获得反馈来学习最佳策略。应用于机器人控制和自动驾驶。


### 关键要点总结

通过以上三个对比鲜明的例子，我们可以看到明确和直接的提示带来的巨大差异：

**模糊提示的问题：**
- 任务不明确：「帮我处理数据」vs「分析销售数据并计算同比增长率」
- 格式不清晰：「给建议」vs「用表格形式输出学习计划」  
- 缺乏上下文：「解释机器学习」vs「为大一学生用简单语言解释」

**明确提示的要素：**
1. **具体的任务描述** - 明确说明要做什么
2. **清晰的输出格式** - 指定表格、列表、字数等要求
3. **明确的上下文** - 说明目标受众和使用场景
4. **具体的约束条件** - 字数限制、格式要求等

记住：**GPT无法读懂你的想法，只能按照你给出的指令执行**。越明确越直接的指令，越能得到符合期望的结果。


如果您想在不更改上述任何内容的情况下实验课程提示，请滚动到课程notebook的最底部访问[**示例演练场**](#example-playground)。

### 练习 2.1 - 西班牙语 {#exercise-21---spanish}
修改`SYSTEM_PROMPT`让GPT用西班牙语输出答案。

In [8]:
# 系统提示 - 这是您应该更改的唯一字段
SYSTEM_PROMPT = "[请替换此文本]"

# 用户提示
PROMPT = "你好GPT，你好吗？"

# 获取GPT回答
response = get_completion(PROMPT, SYSTEM_PROMPT)

# 检查练习答案是否正确
def grade_exercise(text):
    """检查回答中是否包含西班牙语问候语"""
    return "hola" in text.lower()

# 显示GPT回答和评分结果
print(response)
print("\n" + "="*20 + " 练习评分 " + "="*20)
result = grade_exercise(response)
print("✅ 练习完成状态:", "正确 ✅" if result else "需要调整 ❌")

你好！我很好，谢谢你的关心。有什么我可以帮助你的吗？

==================== 练习评分 ====================
✅ 练习完成状态: 需要调整 ❌


❓ 如果您需要提示，请运行下面的单元格！

In [9]:
# 导入并显示练习2.1的提示
from hints import exercise_2_1_hint; print(exercise_2_1_hint)

本练习中的评分函数正在寻找包含"hola"单词的任何答案。
像您与人类交谈时一样要求GPT用西班牙语回复。就是这么简单！


### 练习 2.2 - 仅一名球员 {#exercise-22---one-player-only}

修改`PROMPT`，使GPT完全不含糊其辞，**仅仅**回复一个特定球员的姓名，**不包含任何其他单词或标点符号**。 

In [10]:
# 提示 - 这是您应该更改的唯一字段
PROMPT = "[请替换此文本]"

# 获取GPT回答
response = get_completion(PROMPT)

# 检查练习答案是否正确
def grade_exercise(text):
    """检查回答是否精确匹配"Michael Jordan" """
    return text == "Michael Jordan"

# 显示GPT回答和评分结果
print(response)
print("\n" + "="*20 + " 练习评分 " + "="*20)
result = grade_exercise(response)
print("✅ 练习完成状态:", "正确 ✅" if result else "需要调整 ❌")

当然可以！请告诉我您希望替换的文本内容，以及您想要替换成什么内容。

==================== 练习评分 ====================
✅ 练习完成状态: 需要调整 ❌


❓ 如果您需要提示，请运行下面的单元格！

In [11]:
# 导入并显示练习2.2的提示
from hints import exercise_2_2_hint; print(exercise_2_2_hint)

本练习中的评分函数正在寻找完全匹配"Michael Jordan"。
您会如何要求另一个人这样做？只回复不包含其他单词？只回复姓名而不回复其他任何内容？有几种方法可以解决这个答案。


### 练习 2.3 - 写故事 {#exercise-23---write-a-story}

修改`PROMPT`，使GPT尽可能给出长篇响应。如果您的答案**超过800个单词**，GPT的响应将被评为正确。

In [12]:
# 提示 - 这是您应该更改的唯一字段
PROMPT = "[请替换此文本]"

# 获取GPT回答
response = get_completion(PROMPT)

# 检查练习答案是否正确
def grade_exercise(text):
    """检查回答字数是否达到800词以上"""
    trimmed = text.strip()
    words = len(trimmed.split())
    return words >= 800

# 显示GPT回答和评分结果
print(response)
print("\n" + "="*20 + " 练习评分 " + "="*20)
result = grade_exercise(response)
print("✅ 练习完成状态:", "正确 ✅" if result else "需要调整 ❌")

当然可以！请告诉我您希望替换的具体内容或主题，我会根据您的要求进行修改。

==================== 练习评分 ====================
✅ 练习完成状态: 需要调整 ❌


❓ 如果您需要提示，请运行下面的单元格！

In [13]:
# 导入并显示练习2.3的提示
from hints import exercise_2_3_hint; print(exercise_2_3_hint)

本单元格中的评分函数正在寻找等于或大于800个单词的响应。
因为LLM还不擅长计算单词数，您可能需要超过目标。


### 恭喜！

如果您已经解决了到此为止的所有练习，您就可以进入下一章了。愉快的提示工程！

In [14]:
# 示例1：数据分析任务
# 模糊的提示
PROMPT = "帮我处理一下这个数据"
print("模糊提示的回答:")
print(get_completion(PROMPT))
print("\n" + "="*50 + "\n")

# 明确的提示
PROMPT = """分析以下销售数据，计算每个月的同比增长率：
1月：100万
2月：120万
3月：110万
4月：130万

要求：
- 以表格形式输出结果
- 包含月份、销售额、同比增长率三列
- 同比增长率保留2位小数，用百分比表示"""

print("明确提示的回答:")
print(get_completion(PROMPT))

模糊提示的回答:
当然可以！请提供你需要处理的数据，并告诉我你希望如何处理它。无论是数据清理、分析还是可视化，我都会尽力帮助你。


明确提示的回答:
为了计算每个月的同比增长率，我们需要将当前月的销售额与去年同月的销售额进行比较。然而，您提供的数据仅包含今年的销售额，因此无法直接计算同比增长率。如果您有去年的数据，请提供以便进行计算。

假设您提供了去年的数据，我们可以按照以下步骤进行计算：

1. 计算同比增长率公式：
   \[
   \text{同比增长率} = \left( \frac{\text{今年销售额} - \text{去年销售额}}{\text{去年销售额}} \right) \times 100\%
   \]

2. 以表格形式输出结果。

请提供去年的销售数据以便继续计算。如果没有去年的数据，我们可以计算环比增长率（即与上个月相比的增长率）。如果您希望计算环比增长率，请确认。


In [15]:
# 示例2：学习建议任务
# 模糊的提示
PROMPT = "给我一些Python学习建议"
print("模糊提示的回答:")
print(get_completion(PROMPT))
print("\n" + "="*50 + "\n")

# 明确的提示
PROMPT = """为初学者制定一个Python学习计划。

要求：
1. 分为3个阶段：初级、中级、高级
2. 每个阶段包含：学习时长、核心概念、推荐资源、实战项目
3. 用表格形式呈现
4. 不要添加额外解释，直接给出表格

输出格式示例：
| 阶段 | 学习时长 | 核心概念 | 推荐资源 | 实战项目 |
|------|----------|----------|----------|----------|
| 初级 | ... | ... | ... | ... |"""

print("明确提示的回答:")
print(get_completion(PROMPT))

模糊提示的回答:
学习Python是一项非常有益的技能，无论你是编程新手还是有经验的开发者。以下是一些建议，帮助你有效地学习Python：

1. **设定明确的目标**：首先，明确你学习Python的目的。是为了数据分析、机器学习、Web开发还是自动化任务？根据你的目标选择合适的学习路径。

2. **掌握基础知识**：从Python的基本语法开始，包括变量、数据类型、运算符、条件语句、循环、函数等。可以使用在线教程、书籍或视频课程。

3. **动手实践**：理论学习固然重要，但实践更能巩固知识。尝试编写简单的程序，解决实际问题。可以从一些小项目开始，比如计算器、猜数字游戏等。

4. **使用在线资源**：利用在线平台如Codecademy、Coursera、edX、LeetCode等进行系统学习。这些平台提供结构化的课程和练习题。

5. **参与社区**：加入Python社区，如Stack Overflow、Reddit的r/learnpython、GitHub等。参与讨论、寻求帮助、分享你的项目。

6. **学习Python库**：根据你的目标，学习相关的Python库。例如，数据分析可以学习Pandas和NumPy，机器学习可以学习Scikit-learn和TensorFlow，Web开发可以学习Django和Flask。

7. **阅读代码**：通过阅读他人的代码来提高自己的编程能力。GitHub是一个很好的资源，可以找到各种项目的源代码。

8. **定期练习**：编程技能需要不断练习。每天花一些时间写代码，解决编程挑战或参与开源项目。

9. **解决问题**：尝试解决编程挑战和问题，比如参加在线编程竞赛或完成Kaggle上的数据科学项目。

10. **持续学习**：技术不断发展，保持学习的习惯。关注Python的最新发展和更新，学习新版本的特性。

通过这些方法，你可以逐步提高自己的Python技能，并应用到实际项目中。祝你学习愉快！


明确提示的回答:
| 阶段 | 学习时长 | 核心概念 | 推荐资源 | 实战项目 |
|------|----------|----------|----------|----------|
| 初级 | 4周 | 变量与数据类型、基本运算、条件语句、循环 | 《Python编程：从入门到实践》、Co

In [16]:
# 示例3：概念解释任务
# 容易产生歧义的提示
PROMPT = "解释一下机器学习"
print("歧义提示的回答:")
print(get_completion(PROMPT))
print("\n" + "="*50 + "\n")

# 明确的上下文和要求
PROMPT = """你是一名面向大学一年级计算机专业学生的讲师。用简单易懂的语言解释机器学习。

要求：
1. 用日常生活中的例子来说明概念
2. 避免复杂的数学公式
3. 重点说明机器学习的3个主要类型及其应用场景
4. 控制在200字以内
5. 用1-2-3的序号格式组织内容"""

print("明确提示的回答:")
print(get_completion(PROMPT))

歧义提示的回答:
机器学习是一种通过算法和统计模型使计算机系统能够从数据中自动学习和改进的技术。它是人工智能的一个分支，旨在让计算机系统在没有明确编程指令的情况下进行预测或决策。机器学习的核心思想是利用数据来训练模型，使其能够识别模式、进行分类、预测结果或生成新的数据。

机器学习通常分为以下几类：

1. **监督学习**：在监督学习中，模型从带有标签的数据中学习。标签是已知的输出或结果，模型通过学习输入数据与输出标签之间的关系来进行预测。常见的监督学习算法包括线性回归、逻辑回归、支持向量机和神经网络。

2. **无监督学习**：无监督学习处理没有标签的数据，目标是发现数据中的结构或模式。常见的无监督学习算法包括聚类（如K均值聚类）和降维（如主成分分析）。

3. **半监督学习**：半监督学习结合了监督学习和无监督学习，使用部分标记的数据来提高模型的性能。

4. **强化学习**：强化学习通过与环境的交互来学习策略，以最大化某种奖励。模型通过试错来学习最佳行动策略，常用于机器人控制和游戏AI。

5. **深度学习**：深度学习是机器学习的一个子领域，使用多层神经网络来处理复杂的数据。它在图像识别、自然语言处理和语音识别等领域取得了显著的成功。

机器学习的应用非常广泛，包括推荐系统、图像识别、自然语言处理、金融预测、医疗诊断等。随着数据量的增加和计算能力的提升，机器学习在各个领域的影响力和应用范围不断扩大。


明确提示的回答:
1. 机器学习就像教电脑识别水果。比如，你给电脑看很多苹果和香蕉的图片，并告诉它哪个是苹果，哪个是香蕉。电脑通过这些例子学习如何区分它们。

2. 机器学习有三种主要类型：监督学习、无监督学习和强化学习。监督学习就像老师给学生答案，然后让学生根据答案做题，比如分类邮件是垃圾邮件还是正常邮件。无监督学习则是让电脑自己找规律，比如根据顾客购买习惯分组。强化学习类似于训练宠物，通过奖励和惩罚让它学会做某件事，比如自动驾驶汽车学习如何在道路上行驶。

3. 这些方法在日常生活中应用广泛，如推荐系统、图像识别和游戏AI等。通过不断学习，机器可以更好地完成任务。


In [17]:
# 额外示例：你可以在这里尝试自己的提示
# 尝试对比模糊提示和明确提示的效果

# 你的模糊提示
PROMPT_VAGUE = "写个程序"
print("你的模糊提示:")
print(get_completion(PROMPT_VAGUE))
print("\n" + "="*50 + "\n")

# 你的明确提示  
PROMPT_CLEAR = """写一个Python程序来管理学生成绩。

要求：
1. 创建一个Student类，包含姓名、学号、成绩属性
2. 实现添加学生、计算平均分、查找最高分学生的功能
3. 用字典存储多个学生数据
4. 包含完整的错误处理
5. 添加详细的注释说明"""

print("你的明确提示:")
print(get_completion(PROMPT_CLEAR))

你的模糊提示:
当然！请告诉我您想要编写什么样的程序，以及使用哪种编程语言。我会尽力帮助您。


你的明确提示:
下面是一个简单的Python程序，用于管理学生成绩。该程序包含一个`Student`类，并实现了添加学生、计算平均分、查找最高分学生的功能。学生数据存储在字典中，并包含了错误处理和详细的注释说明。

```python
class Student:
    def __init__(self, name, student_id, score):
        """
        初始化Student类的实例。

        :param name: 学生姓名
        :param student_id: 学生学号
        :param score: 学生成绩
        """
        self.name = name
        self.student_id = student_id
        self.score = score

def add_student(students_dict, name, student_id, score):
    """
    添加学生到字典中。

    :param students_dict: 存储学生数据的字典
    :param name: 学生姓名
    :param student_id: 学生学号
    :param score: 学生成绩
    :return: None
    """
    try:
        if student_id in students_dict:
            raise ValueError("学号已存在，请使用唯一的学号。")
        students_dict[student_id] = Student(name, student_id, score)
    except ValueError as e:
        print(f"错误: {e}")

def calculate_average_score(students_dict):
    """
    计算所有学生的平均分。

    :param students_dict: 存储学生数据的字典
    